In [1]:
from pymongo import MongoClient
import feedparser
from pprint import pprint
import datetime
import random

def build_heading_quiz(answers):

    # open database connection
    username="root"
    password="mongo"
    client = MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
    db = client.headings
    
    # loop for 10 attempts
    midnight = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time())
    sources_all = [x["name"] for x in db.sources.find()]
    for _ in range(10):
    
        # pick random sources
        sources = random.sample(sources_all, answers)
        random.shuffle(sources)
        
        # get heading for one of the sources
        heading_source_name = None
        for s in sources:
            source_headings = list(db.headings.aggregate(
                [
                    {"$match": {"_timestamp": {"$gte": midnight}}},
                    {"$sample": {"size": 1} }
                ]
            )
                                  )
            if source_headings and len(source_headings) > 0:
                heading = source_headings[0]
                heading_source_name = s
                break
                
        # did we get an answer?
        if heading_source_name:
            random.shuffle(sources)
            index = sources.index(heading_source_name)
            return {"title": heading["title"], "sources": sources, "index": index, "link": heading["link"]}
    
    # if here, not found
    return None

In [2]:
build_heading_quiz(4)

{'title': "‘Catastrophe for human rights’ as Greece steps up refugee ‘pushbacks'",
 'sources': ['Metro', 'Channel 4 News', 'The Sun', 'Daily Express'],
 'index': 2,
 'link': 'https://www.theguardian.com/global-development/2020/sep/27/catastrophe-for-human-rights-as-greece-steps-up-refugee-pushbacks'}

In [3]:
import logging
import os

from telegram import (Poll, ParseMode, KeyboardButton, KeyboardButtonPollType,
                      ReplyKeyboardMarkup, ReplyKeyboardRemove)
from telegram.ext import (Updater, CommandHandler, PollAnswerHandler, PollHandler, MessageHandler,
                          Filters)

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


def start(update, context):
    """Inform user about what this bot can do"""
    update.message.reply_text('Please select /quiz, /quiz3 or /quiz4 to get a Quiz')
    
def quiz3(update, context):
    quiz(update, context, 3)

def quiz4(update, context):
    quiz(update, context, 4)

    
def quiz(update, context, qs=2):
    
    q = build_heading_quiz(qs)
    
    """Send a predefined poll"""
    questions = q["sources"]
    message = update.effective_message.reply_poll("Where was this heading published?\n\n" + q["title"],
                                                  questions, type=Poll.QUIZ, correct_option_id=q["index"])
    # Save some info about the poll the bot_data for later use in receive_quiz_answer
    payload = {message.poll.id: {"chat_id": update.effective_chat.id,
                                 "message_id": message.message_id,
                                 "q": q,
                                 "qs": qs}}
    context.bot_data.update(payload)

def post_quiz(context):
    pprint(context)
    """Send the alarm message."""
    quiz_data = context.job.context
    context.bot.send_message(quiz_data["chat_id"], text=quiz_data["q"]["link"] + "\n\n" + "/quiz, /quiz3 or /quiz4")  
            

def receive_quiz_answer(update, context):
    pprint(update)
    pprint(context)
    pprint(update.poll)
    pprint(update.poll_answer)
    pprint(context.bot_data)
    
    """Close quiz after three participants took it"""
    # the bot can receive closed poll updates we don't care about
    if update.poll.is_closed:
        return
    if update.poll.total_voter_count == 3:
        try:
            quiz_data = context.bot_data[update.poll.id]
        # this means this poll answer update is from an old poll, we can't stop it then
        except KeyError:
            return
        context.bot.stop_poll(quiz_data["chat_id"], quiz_data["message_id"])
        
    if update.poll:
        pprint("what's next")
        quiz_data = context.bot_data[update.poll.id]
        chat_id = quiz_data["chat_id"]
        new_job = context.job_queue.run_once(post_quiz, 2, context=quiz_data)

        
def help_handler(update, context):
    """Display a help message"""
    update.message.reply_text("Use /quiz, /quiz3 or /quiz4 to test this "
                              "bot.")


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater(os.environ['TEL_BOT_TOKEN'], use_context=True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('start', start))
    dp.add_handler(CommandHandler('quiz', quiz))
    dp.add_handler(CommandHandler('quiz3', quiz3))
    dp.add_handler(CommandHandler('quiz4', quiz4))
    dp.add_handler(PollHandler(receive_quiz_answer))
    dp.add_handler(CommandHandler('help', help_handler))

    # Start the Bot
    updater.start_polling()

    # Run the bot until the user presses Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT
    updater.idle()

In [ ]:
main()

None
{'6019357323443568645': {'chat_id': 327434945,
                         'message_id': 284,
                         'q': {'index': 0,
                               'link': 'https://www.thesun.co.uk/sport/football/12776550/tottenham-newcastle-live-stream-tv-watch-premier-league-online/',
                               'sources': ['The Independent', 'Channel 4 News'],
                               'title': 'Tottenham 1 Newcastle 1 LIVE '
                                        'REACTION: Spurs FUME at VAR as Toon '
                                        'awarded controversial stoppage-time '
                                        'penalty'},
                         'qs': 2}}
"what's next"
None
{'6019357323443568645': {'chat_id': 327434945,
                         'message_id': 284,
                         'q': {'index': 0,
                               'link': 'https://www.thesun.co.uk/sport/football/12776550/tottenham-newcastle-live-stream-tv-watch-premier-league-online/',


None
{'6019357323443568645': {'chat_id': 327434945,
                         'message_id': 284,
                         'q': {'index': 0,
                               'link': 'https://www.thesun.co.uk/sport/football/12776550/tottenham-newcastle-live-stream-tv-watch-premier-league-online/',
                               'sources': ['The Independent', 'Channel 4 News'],
                               'title': 'Tottenham 1 Newcastle 1 LIVE '
                                        'REACTION: Spurs FUME at VAR as Toon '
                                        'awarded controversial stoppage-time '
                                        'penalty'},
                         'qs': 2},
 '6019357323443568646': {'chat_id': 327434945,
                         'message_id': 287,
                         'q': {'index': 1,
                               'link': 'https://metro.co.uk/2020/09/27/93-year-old-cyclist-still-clocks-up-over-150-miles-a-week-13333936/',
                               's

None
{'6019357323443568645': {'chat_id': 327434945,
                         'message_id': 284,
                         'q': {'index': 0,
                               'link': 'https://www.thesun.co.uk/sport/football/12776550/tottenham-newcastle-live-stream-tv-watch-premier-league-online/',
                               'sources': ['The Independent', 'Channel 4 News'],
                               'title': 'Tottenham 1 Newcastle 1 LIVE '
                                        'REACTION: Spurs FUME at VAR as Toon '
                                        'awarded controversial stoppage-time '
                                        'penalty'},
                         'qs': 2},
 '6019357323443568646': {'chat_id': 327434945,
                         'message_id': 287,
                         'q': {'index': 1,
                               'link': 'https://metro.co.uk/2020/09/27/93-year-old-cyclist-still-clocks-up-over-150-miles-a-week-13333936/',
                               's

None
{'6019357323443568645': {'chat_id': 327434945,
                         'message_id': 284,
                         'q': {'index': 0,
                               'link': 'https://www.thesun.co.uk/sport/football/12776550/tottenham-newcastle-live-stream-tv-watch-premier-league-online/',
                               'sources': ['The Independent', 'Channel 4 News'],
                               'title': 'Tottenham 1 Newcastle 1 LIVE '
                                        'REACTION: Spurs FUME at VAR as Toon '
                                        'awarded controversial stoppage-time '
                                        'penalty'},
                         'qs': 2},
 '6019357323443568646': {'chat_id': 327434945,
                         'message_id': 287,
                         'q': {'index': 1,
                               'link': 'https://metro.co.uk/2020/09/27/93-year-old-cyclist-still-clocks-up-over-150-miles-a-week-13333936/',
                               's